In [1]:
#Import the necessary packages
import pandas as pd
from sklearn.preprocessing import StandardScaler

colnames = ['number_preg','pgc','dbp','tsft','si','bmi','dpf','age','class']
#Load The dataset
data = pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.csv', names = colnames)

#Remove datasets with question marks
data = data[(data != '?').all(axis=1)]

#Show top rows
data.head()

/srv/conda/envs/notebook/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,number_preg,pgc,dbp,tsft,si,bmi,dpf,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [2]:
#Define X and y
X= data.drop('class', axis = 1)
y = data[['class']]
X.info()
print('\n')
y.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 768 entries, 0 to 767
Data columns (total 8 columns):
number_preg    768 non-null int64
pgc            768 non-null int64
dbp            768 non-null int64
tsft           768 non-null int64
si             768 non-null int64
bmi            768 non-null float64
dpf            768 non-null float64
age            768 non-null int64
dtypes: float64(2), int64(6)
memory usage: 54.0 KB


<class 'pandas.core.frame.DataFrame'>
Int64Index: 768 entries, 0 to 767
Data columns (total 1 columns):
class    768 non-null int64
dtypes: int64(1)
memory usage: 12.0 KB


In [3]:
#Normalize feature variables
sc = StandardScaler()
X = pd.DataFrame(sc.fit_transform(X), columns=X.columns)
X.head()

,number_preg,pgc,dbp,tsft,si,bmi,dpf,age
0,0.639947,0.848324,0.149641,0.907270,-0.692891,0.204013,0.468492,1.425995
1,-0.844885,-1.123396,-0.160546,0.530902,-0.692891,-0.684422,-0.365061,-0.190672
2,1.233880,1.943724,-0.263941,-1.288212,-0.692891,-1.103255,0.604397,-0.105584
3,-0.844885,-0.998208,-0.160546,0.154533,0.123302,-0.494043,-0.920763,-1.041549
4,-1.141852,0.504055,-1.504687,0.907270,0.765836,1.409746,5.484909,-0.020496


In [5]:
!pip install Keras
# Create the function that returns the keras model
from keras.models import Sequential
from keras.layers import Dense
from keras.regularizers import l2

#define build_model function
def build_model(lambda_parameter):
    model = Sequential()
    model.add(Dense(10, input_dim=X.shape[1], activation='relu', kernel_regularizer=l2(lambda_parameter)))
    model.add(Dense(6, activation='relu', kernel_regularizer=l2(lambda_parameter)))
    model.add(Dense(4, activation='relu', kernel_regularizer=l2(lambda_parameter)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])
    return model

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
     |████████████████████████████████| 377 kB 3.4 MB/s eta 0:00:01


Using TensorFlow backend.
/srv/conda/envs/notebook/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/srv/conda/envs/notebook/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/srv/conda/envs/notebook/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/srv/conda/envs/notebook/lib/python3.7/site-packag

In [7]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

# define a seed for random number generator so the result will be reproducible
import numpy as np
import tensorflow as tf

seed = 1
np.random.seed(seed)
tf.set_random_seed(seed)

# create the Keras wrapper with scikit learn
model = KerasClassifier(build_fn=build_model, verbose=0)

# define all the possible values for each hyperparameter
lambda_parameter = [0.01, 0.5, 1]
epochs = [50, 100]
batch_size = [20]

# create the dictionary containing all possible values of hyperparameters
param_grid = dict(lambda_parameter=lambda_parameter, epochs=epochs, batch_size=batch_size)

# perform 5-fold cross validation 
grid_seach = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
results_1 = grid_seach.fit(X, y)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [8]:
# print the results for best cross validation score
print("Best cross validation score =", results_1.best_score_)
print("Parameters for Best cross validation scor e=", results_1.best_params_)

# print the results for all evaluated hyperparameter combinations
accuracy_means = results_1.cv_results_['mean_test_score']
accuracy_stds = results_1.cv_results_['std_test_score']
parameters = results_1.cv_results_['params']
for p in range(len(parameters)):
    print("Accuracy %f (std %f) for params %r" % (accuracy_means[p], accuracy_stds[p], parameters[p]))

Best cross validation score = 0.7695312485254059
Parameters for Best cross validation scor e= {'batch_size': 20, 'epochs': 100, 'lambda_parameter': 0.01}
Accuracy 0.740885 (std 0.056878) for params {'batch_size': 20, 'epochs': 50, 'lambda_parameter': 0.01}
Accuracy 0.651042 (std 0.052404) for params {'batch_size': 20, 'epochs': 50, 'lambda_parameter': 0.5}
Accuracy 0.651042 (std 0.052404) for params {'batch_size': 20, 'epochs': 50, 'lambda_parameter': 1}
Accuracy 0.769531 (std 0.039153) for params {'batch_size': 20, 'epochs': 100, 'lambda_parameter': 0.01}
Accuracy 0.651042 (std 0.052404) for params {'batch_size': 20, 'epochs': 100, 'lambda_parameter': 0.5}
Accuracy 0.651042 (std 0.052404) for params {'batch_size': 20, 'epochs': 100, 'lambda_parameter': 1}


In [9]:
# define a seed for random number generator so the result will be reproducible
np.random.seed(seed)
tf.set_random_seed(seed)

# create the Keras wrapper with scikit learn
model = KerasClassifier(build_fn=build_model, verbose=0)

# define all the possible values for each hyperparameter
lambda_parameter = [0.001, 0.01, 0.05, 0.1]
epochs = [100]
batch_size = [20]

# create the dictionary containing all possible values of hyperparameters
param_grid = dict(lambda_parameter=lambda_parameter, epochs=epochs, batch_size=batch_size)

# search the grid, perform 5-fold cross validation for each possible combination, store the results
grid_seach = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
results_2 = grid_seach.fit(X, y)

In [10]:
# print the results for best cross validation score
print("Best cross validation score =", results_2.best_score_)
print("Parameters for Best cross validation score =", results_2.best_params_)

# print the results for the entire grid
accuracy_means = results_2.cv_results_['mean_test_score']
accuracy_stds = results_2.cv_results_['std_test_score']
parameters = results_2.cv_results_['params']
for p in range(len(parameters)):
    print("Accuracy %f (std %f) for params %r" % (accuracy_means[p], accuracy_stds[p], parameters[p]))

Best cross validation score = 0.7695312490686774
Parameters for Best cross validation score = {'batch_size': 20, 'epochs': 100, 'lambda_parameter': 0.01}
Accuracy 0.764323 (std 0.039638) for params {'batch_size': 20, 'epochs': 100, 'lambda_parameter': 0.001}
Accuracy 0.769531 (std 0.038347) for params {'batch_size': 20, 'epochs': 100, 'lambda_parameter': 0.01}
Accuracy 0.730469 (std 0.061362) for params {'batch_size': 20, 'epochs': 100, 'lambda_parameter': 0.05}
Accuracy 0.651042 (std 0.052404) for params {'batch_size': 20, 'epochs': 100, 'lambda_parameter': 0.1}


In [11]:
# Create the function that returns the keras model
from keras.layers import Dropout
def build_model(rate):
    model = Sequential()
    model.add(Dense(10, input_dim=X.shape[1], activation='relu'))
    model.add(Dropout(rate))
    model.add(Dense(6, activation='relu'))
    model.add(Dropout(rate))
    model.add(Dense(4, activation='relu'))
    model.add(Dropout(rate))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])
    return model

In [12]:
# define a seed for random number generator so the result will be reproducible
np.random.seed(seed)
tf.set_random_seed(seed)

# create the Keras wrapper with scikit learn
model = KerasClassifier(build_fn=build_model, verbose=0)

# define all the possible values for each hyperparameter
rate = [0, 0.1, 0.2]
epochs = [50, 100]
batch_size = [20]

# create the dictionary containing all possible values of hyperparameters
param_grid = dict(rate=rate, epochs=epochs, batch_size=batch_size)

# perform 5-fold cross validation for 10 randomly selected combinations, store the results
grid_seach = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
results_3 = grid_seach.fit(X, y)

In [13]:
# print the results for best cross validation score
print("Best cross validation score=", results_3.best_score_)
print("Parameters for Best cross validation score=", results_3.best_params_)

# print the results for the entire grid
accuracy_means = results_3.cv_results_['mean_test_score']
accuracy_stds = results_3.cv_results_['std_test_score']
parameters = results_3.cv_results_['params']
for p in range(len(parameters)):
    print("Accuracy %f (std %f) for params %r" % (accuracy_means[p], accuracy_stds[p], parameters[p]))

Best cross validation score= 0.7682291663562258
Parameters for Best cross validation score= {'batch_size': 20, 'epochs': 100, 'rate': 0.1}
Accuracy 0.735677 (std 0.047776) for params {'batch_size': 20, 'epochs': 50, 'rate': 0}
Accuracy 0.730469 (std 0.028613) for params {'batch_size': 20, 'epochs': 50, 'rate': 0.1}
Accuracy 0.695313 (std 0.069350) for params {'batch_size': 20, 'epochs': 50, 'rate': 0.2}
Accuracy 0.768229 (std 0.046477) for params {'batch_size': 20, 'epochs': 100, 'rate': 0}
Accuracy 0.768229 (std 0.034960) for params {'batch_size': 20, 'epochs': 100, 'rate': 0.1}
Accuracy 0.699219 (std 0.079999) for params {'batch_size': 20, 'epochs': 100, 'rate': 0.2}
